In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import Add

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Convolution with Batch Norm & ReLU

In [2]:
inputs = tf.keras.Input(shape=784,) name="digits")

def convolution_BNReLU(filters, kernel_size, strides = 1):
    ConvBNReLU = Sequential()
    
    # Conv2D 
    ConvBNReLU.add(Conv2D(filters, kernel_size, strides, padding='valid'))
    # Batch Normalization
    ConvBNReLU.add(BatchNormalization())
    # Activation Function
    ConvBNReLU.add(Activation("relu"))            
    
    return ConvBNReLU  

# Depthwise Separable Convolution

In [3]:
def depthwise_separable(filters, kernel_size, strides = 1):
    DWSeparableConv = Sequential()
    
    # Depthwise Convolution 3x3
    DWSeparableConv.add(DepthwiseConv1D(filters, kernel_size, strides, padding='valid'))
    # Batch Normalization
    DWSeparableConv.add(BatchNormalization())
    # Activation Function
    DWSeparableConv.add(Activation("relu"))            
    
    return DWSeparableConv  

# Separable Squeeze-Expand Block

In [4]:
def SSE_Block(squeeze, expand):
    # Squeeze Layer - pointwise
    pointwise1 = convolution_BNReLU(squeeze, 1)
    
    # Expand Layer - pointwise followed by depthwise
    pointwise2 = convolution_BNReLU(expand, 1)
    depthwise = convolution_BNReLU(expand, 3)            

    #Left side of SSE: pointwise to pointwise
    left = Sequential()
    left.add(pointwise1)
    left.add(depthwise)

    #right side of SSE: pointwise to depthwise
    right = Sequential()
    right.add(pointwise1)
    right.add(depthwise)
    
    SSE = concatenate(axis=1)([left, right])
    return SSE

# Slim Module

In [5]:

def SlimModule(squeeze, expand, dws):
    step1 = SSE_Block(squeeze, expand)(i)
    
    step2 = Add()([step1, i])
    
    step3 = SSE_Block(squeeze, expand)(step2)
    
    pool = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(step3)

# SlimNet - CNN

In [6]:
inputShape = (178, 218, 3)

SlimNetModel = Sequential()

SlimNetModel.add(Conv2D(96, kernel_size=(7,7),
                       strides=(2,2),
                       input_shape=inputShape))
SlimNetModel.add(MaxPool2D(pool_size=(3, 3),
                             strides=(2, 2)))

vector = [1,2,3,4]
for n in vector:
    squeeze = 16 * n
    expand = 64 * n
    dws = 48 * n
    SlimNetModel.add(SlimModule(squeeze, expand, dws))

TypeError: concatenate() missing 1 required positional argument: 'inputs'